# Implementation for Task1

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from typing import Callable

We solve the diffusion-reaction equation

$$
u_t = \mu u_xx + a u
$$

where $\mu$ is a positive constant and $a$ is a constant.

The domain is $(x, t) \in [0, 1] \times [0, T]$, where $T$ is the end point in time.
Boundary conditions are given by functions $f, g_1$ and $g_2$:

\begin{align*}
u(0, x) &= f(x) \\
u(t, 0) &= g_1(t) \\
u(t, 1) &= g_2(t)
\end{align*}

The following class implements a solver based on the discretization in the project description.

In [ ]:
class ReacDiffSolver:
    """Solving the linear diffusion-reaction equation

    u_t = mu u_xx + a u

using a modified Crank Nicholson scheme on the domain
0 <= x <= 1, 0 <= t <= T, with boundary conditions
u(0, x) = f(x), u(t, 0) = g1(t), u(t, 1) = g2(t).
    """

    def __init__(self, mu: float, a: float, f: Callable, g1: Callable, g2: Callable, alpha: float = 1) -> None:
        self.mu = mu
        self.a = a
        self.f = f
        self.g1 = g1
        self.g2 = g2
        self.alpha = alpha

    def solve(self, M: int, N: int, T: int=1) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
        """M: number of inner grid points in space.
        N: number of steps in time.
        T: end time

        returns:
            array of size (N+1, M+2)
            array with points in space (M+2)
            array with points in time (N+1)
        """
        k = 1/N                 # step size in time
        h = 1/(M + 1)           # step size in space
        r = self.mu * k / h**2

        sol = np.zeros((N + 1, M+2))
        x = np.linspace(0, 1, M + 2)
        t = np.linspace(0, 1, N + 1)

        # set boundaries
        sol[0, :] = self.f(x)
        sol[1:, 0] = self.g1(t[1:])
        sol[1:, -1] = self.g2(t[1:])

        # construct the lhs of the system: A U* = b
        A = sp.sparse.diags(
            [-r/2, 1 + r, -r/2],
            offsets=[-1, 0, 1],
            shape=(M, M)
        ).tocsr()

        for n in range(N):
            Unm1 = sol[n, :-2]  # U_{n-1}
            Un = sol[n, 1:-1]   # U_{n}
            Unp1 = sol[n, 2:]   # U_{n+1}

            # solve equation for U* (implicit step)
            b = (1 + k*self.a)*Un + r/2*(Unm1 - 2*Un + Unp1)
            b[0] += r/2 * self.g1(t[n] + self.alpha*k)     # how to choose U_0^*?
            b[-1] += r/2 * self.g2(t[n] + self.alpha*k)    # how to choose U_{M+1}^*?

            Ustar = sp.sparse.linalg.spsolve(A, b)

            # do a step forward in time (explicit step)
            sol[n + 1, 1:-1] = Ustar + k/2*(self.a*Ustar - self.a*Un)

        return sol, x, t

In [ ]:
def anal(t, x, mu, a, b, phi):
    return np.exp((-mu*b**2 + a)*t)*np.sin(b*x + phi)

def plot3d(x, t, sol):
    fig = plt.figure(figsize=(16, 10))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(*np.meshgrid(x, t), sol)
    ax.set_xlabel('x')
    ax.set_ylabel('t')
    fig.savefig("plots/3dplot.pdf")

In [ ]:
mu = 1/5
a = 1
b = 3/2*np.pi
phi = np.pi/4

solver = ReacDiffSolver(
    mu=mu,
    a=a,
    f=lambda x: anal(0, x, mu, a, b, phi),
    g1=lambda t: anal(t, 0, mu, a, b, phi),
    g2=lambda t: anal(t, 1, mu, a, b, phi),
    alpha=1
)

sol, x, t = solver.solve(M=100, N=101)

plot3d(x, t, sol)


Let's try to take max norm over both space and time, and let $h \to 0$ whilst fixating $k$.

In [ ]:
N_const = M_const = 10_000

NUM_POINTS = 5
Ms = np.logspace(1, 3, NUM_POINTS, dtype=int)
Ns = np.logspace(1, 3, NUM_POINTS, dtype=int)

errs_M = np.zeros(Ms.size, dtype=float)
errs_N = np.zeros(Ns.size, dtype=float)
errs_NM = np.zeros(Ns.size, dtype=float)

for i in range(NUM_POINTS):
    # keep k constant and let h -> 0
    sol1, x1, t1 = solver.solve(M=Ms[i], N=N_const)
    _anal1 = anal(*np.meshgrid(t1, x1), mu, a, b, phi).T
    errs_M[i] = np.max(
        1/np.sqrt(x1.size)*np.linalg.norm(sol1 - _anal1, ord=2, axis=1)
    )

    # keep h constant and let k -> 0
    sol2, x2, t2 = solver.solve(M=M_const, N=Ns[i])
    _anal2 = anal(*np.meshgrid(t2, x2), mu, a, b, phi).T
    errs_N[i] = np.max(
        1/np.sqrt(x2.size)*np.linalg.norm(sol2 - _anal2, ord=2, axis=1)
    )

    # let h = k -> 0
    sol3, x3, t3 = solver.solve(M=Ms[i], N=Ns[i])
    _anal3 = anal(*np.meshgrid(t3, x3), mu, a, b, phi).T
    errs_NM[i] = np.max(
        1/np.sqrt(x3.size)*np.linalg.norm(sol3 - _anal3, ord=2, axis=1)
    )


In [ ]:
fig1, axs = plt.subplots(1, 2)

axs[0].loglog(1/(1 + Ms), (lambda x: x**2)(1/(1+Ms)), '--', label="$h^2$")
axs[0].loglog(1/(1 + Ms), errs_M, '.-b', label=f"k = {1/N_const}")
axs[0].set(
    xlabel=r"$h$",
    ylabel=r"$||E^n||_{2,h}$",
    title=r"Vary $h$, keep $k$ constant"
)

axs[1].loglog(1/(Ns), (lambda x: x**2)(1/Ns), '--', label="$k^2$")
axs[1].loglog(1/Ns, errs_N, '.-b', label=f"h = {1/M_const}")
axs[1].set(
    xlabel=r"$k$",
    ylabel=r"$||E^n||_{2,h}$",
    title=r"Vary $k$, keep $h$ constant"
)
fig1.suptitle("Global error")
axs[0].legend()
axs[1].legend()

fig1.tight_layout()
fig1.savefig("plots/task1_err1.pdf")

fig2, ax = plt.subplots()
ax.loglog(1/Ns, (lambda x: x**2)(1/Ns), '--', label="$h^2$")
ax.loglog(1/Ns, (lambda x: x)(1/Ns), '--', label="$h$")
ax.loglog(1/Ns, errs_NM, '.-b', label="")
ax.set(
    xlabel=r"$h=k$",
    ylabel=r"$||E^n||_{2,h}$",
    title=r"Convergence plot. $(h,k) \to (0,0)$ along $h = k$."
)
ax.legend()
fig2.tight_layout()
fig2.savefig("plots/task1_err2.pdf")